In [ ]:
def try_to_mount_drive():
    """Mount Google Drive. For local usage only."""
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        import os
        os.chdir('/content/drive/MyDrive/Colab Notebooks')
        return True
    except ImportError:
        print("Google Colab module not found. Skipping Google Drive mount.")
        return False

In [ ]:
def read_data(filename):
    """Read the iris dataset with the correct column names."""
    import pandas as pd
    data = pd.read_csv(filename, header=None)
    data.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
    return data

In [ ]:
def encode_labels_to_numerical(data):
    """Encode the labels to numerical values. eg. iris_color to 0."""
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    data['species'] = label_encoder.fit_transform(data['species'])
    return data, label_encoder.classes_

In [ ]:
def encode_labels_as_one_hot(y, num_classes=3):
    """Encodes labels to one-hot format. Use keras.utils to_categorical function for this."""
    from keras.utils import to_categorical
    return to_categorical(y, num_classes=num_classes)

In [ ]:
def separate_features_and_labels(data):
    """Split the dataset into features and labels."""
    features = data.drop('species', axis=1).values
    labels = data['species'].values
    return features, labels

In [ ]:
def split_data_to_test_and_train(X, y):
    """Split the dataset into two for testing and validation. Choose the proper split."""
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
def create_model():
    """Create a model with different layers."""
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    model = Sequential()
    model.add(Dense(10, input_dim=4, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    return model

In [ ]:
def compile_model(model):
    """Use the model.compile function to compile the model with a loss function, optimizer and metrics."""
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, X_train, X_test, y_train, y_test, epochs=150):
    """Train the the model using training and testing datasets."""
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=5, validation_data=(X_test, y_test), verbose=1)
    return history

In [ ]:
def evaluate_model(model, X_test, y_test):
    """Evaluate the model using testing dataset. Use the built-in model.evaluate function to find and return the accuracy."""
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

In [ ]:
def evaluate_using_predictions(model, X_test, y_test):
    """Evaluate the model using testing dataset. Use the built-in model.predict function and compare the predictions against the actual values. Return the accuracy value."""
    import numpy as np
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Convert one-hot encoded y_test to class indices
    actual_classes = np.argmax(y_test, axis=1)
    
    # Now both arrays are shape (30,)
    accuracy = np.mean(predicted_classes == actual_classes)
    return accuracy

In [ ]:
def main():
    running_locally = try_to_mount_drive()

    # This is necessary for testing purposes.
    if not running_locally:
        return
    
    # Proceed with logic
    data = read_data('iris.data')
    encode_labels_to_numerical(data)
    X, y = separate_features_and_labels(data)

    # Encode labels to one-hot vectors
    y = encode_labels_as_one_hot(y, num_classes=3)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = split_data_to_test_and_train(X, y)

    # Model creation, compilation, and training
    model = create_model()
    compile_model(model)
    history = train_model(model, X_train, X_test, y_train, y_test)

    # Model evaluation
    accuracy = evaluate_model(model, X_test, y_test)
    print('Accuracy: %.2f' % (accuracy * 100))

    prediction_accuracy = evaluate_using_predictions(model, X_test, y_test)
    print(f'Total accuracy: {prediction_accuracy * 100:.2f}%')

    # Print model configuration
    config = model.get_config()
    print(config)
    